<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Upside_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

print_date_time()

gen_output = 0

Run date time (IST): 2024-07-14 15:35:50


In [44]:
# install libraries
!pip install yfinance
!pip install pandas-ta
!pip install pyxirr
#!pip install talib-binary

In [45]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [46]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [47]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, how='right', on="Symbol")

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_mymmtm_df():
  mymmtm_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspects-Momentum.csv')
  return mymmtm_df

def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS', 'UJJIVAN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

mymmtm_df = get_mymmtm_df()

df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
#cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','Breakout','StarStock', 'Conviction', 'Category', 'Stock', 'MedianPE']
#df_mypf = df_mypf[cols]
my_prospects = get_stock_ids(df_mypf)
df_stocks = df_mypf[df_mypf['Symbol'].isin(my_prospects)]

In [48]:
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

In [49]:
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category']
df_stocks = df_stocks[common_cols]

In [50]:

# function to get prospects technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)

    stock_df['Min'] = round(min(stock_df['Close']),2)
    #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    #stock_df['Close'] = round(stock_df['Close'],0)
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to get common features for prospects
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)

  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, mymmtm_df, on= 'Symbol', how='left')
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  return df_prec_dev

stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

In [51]:
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [52]:
# prospects stocks

def get_prospects_features(df_common_features):

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  #tmp_df = df_common_features[df_common_features['Stock'].isin(df_mypf[df_mypf['InPortfolio'] != 'NA'].Stock.values)]
  tmp_df = df_common_features
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Today%'] = round((tmp_df['Close'] - tmp_df['Prev_Close'])*100/tmp_df['Prev_Close'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']

  return tmp_df

df_prospects_features = get_prospects_features(df_common_features)

qualified stocks: 163
with latest results: 152
still star stocks: 90


In [53]:
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Momentum', 'Today%', 'FTT%', 'Gained%', 'Criteria', 'Strategy','LatestQtr', 'StarStock', 'Conviction', 'Category']
df_prospects_features = df_prospects_features[cols]

if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prospects_features.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [54]:
# avsp
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AVSP'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
7,AMBUJACEM,676.50,733.0,23.22,70.00,UP,-0.39,8.35,66.86,AVSP,BTT,0,0,H,CEMENT
88,JSWHL,7056.35,7487.0,18.97,68.11,UP,-0.25,6.10,61.18,AVSP,BTT,1,0,H,FINANCE


In [55]:
# av40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AV40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
83,ITC,459.05,576.00,7.51,28.79,NaN,0.09,25.48,15.83,AV40,BTT,1,1,H,FMCG
59,GILLETTE,7363.35,8980.00,11.65,1.79,UP,-0.27,21.96,37.59,AV40,BTT,0,0,H,FMCG
116,PFIZER,4856.45,5855.00,12.97,10.62,UP,-1.91,20.56,27.28,AV40,NTT,1,0,H,PHARMA
147,TITAN,3229.85,3854.04,-7.13,-5.33,NaN,-0.58,19.33,11.66,AV40,ATH,1,1,H,JEWELLERY
5,AKZOINDIA,2885.00,3300.00,13.00,-9.94,UP,-0.42,14.38,26.19,AV40,NTT,1,1,H,PAINTS
117,PGHH,16965.15,18360.87,1.30,-5.98,NaN,0.30,8.23,12.63,AV40,ATH,0,0,H,FMCG
18,BAJAJ-AUTO,9430.75,9961.75,25.49,75.38,UP,-0.38,5.63,107.40,AV40,ATH,1,1,H,AUTO
60,GLAXO,2570.70,2700.70,29.31,33.38,UP,0.83,5.06,87.22,AV40,ATH,1,0,H,PHARMA
159,WHIRLPOOL,1987.15,2081.00,32.57,53.49,UP,-1.22,4.72,62.69,AV40,NTT,1,0,H,DURABLES
107,NESTLEIND,2609.00,2729.54,5.16,-1.19,UP,0.61,4.62,21.45,AV40,ATH,0,0,H,FMCG


In [56]:
# av40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AV40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
101,MOTILALOFS,532.50,1342.00,-23.05,-71.70,DOWN,-0.08,152.02,73.18,AV40N,NTT,1,1,M,FINANCE
85,JCHAC,1942.80,3219.25,51.90,NaN,UP,1.31,65.70,106.49,AV40N,ATH,1,0,M,AC
156,VIPIND,465.90,740.05,-16.51,NaN,NaN,-0.52,58.84,2.87,AV40N,ATH,1,0,M,MISC
51,EQUITASBNK,91.61,114.55,-6.52,-21.78,NaN,0.34,25.04,10.98,AV40N,ATH,1,1,M,BANKS
100,MCX,3771.35,4167.80,15.47,498.71,UP,0.32,10.51,146.01,AV40N,ATH,1,0,M,MISC
124,RADICO,1668.65,1832.95,5.88,104.60,UP,0.18,9.85,44.05,AV40N,ATH,1,1,M,MISC
120,POLYCAB,6581.00,7227.33,21.26,50.95,UP,-0.57,9.82,70.46,AV40N,ATH,1,1,M,CABLES
137,SYMPHONY,1118.50,1201.00,18.86,-4.84,UP,-0.81,7.38,35.95,AV40N,NTT,1,0,M,DURABLES
81,ISEC,776.30,822.10,9.34,-16.38,UP,-1.10,5.90,33.46,AV40N,ATH,1,1,M,FINANCE
35,CERA,8988.70,9483.65,14.14,21.73,NaN,0.74,5.51,36.47,AV40N,ATH,1,1,M,CERAMICS


In [57]:
# av200, avr and aar, exp50
df_tmp = df_prospects_features[~df_prospects_features['Criteria'].isin(['AV40','AV40N'])].sort_values(by = 'FTT%', ascending=False)
df_tmp = df_tmp[df_tmp['Criteria'].notnull()]
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
149,TV18BRDCST,43.22,211.63,-11.80,-949.75,DOWN,0.42,389.66,9.97,AVR,ATH,1,0,M,ENTERTAINMENT
34,CENTRALBK,63.32,190.06,11.09,39.38,UP,0.17,200.16,111.77,AVR,ATH,1,1,M,BANKS
29,CAMPUS,294.25,631.85,10.21,6.91,DOWN,0.15,114.73,37.18,AAR,ATH,1,0,L,FOOTWEAR
48,DREAMFOLKS,475.35,815.64,-7.16,NaN,NaN,-0.62,71.59,5.59,EXP50,ATH,1,1,L,MISC
11,ARVINDFASN,509.45,784.00,16.85,NaN,UP,-1.84,53.89,70.03,AVR,ATH,1,0,L,MISC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,CASTROLIND,251.58,172.00,41.34,67.95,UP,0.78,-31.63,102.22,AVR,NTT,0,0,M,CHEMICALS
12,ARE&M,1615.75,1066.00,69.90,88.00,NaN,-2.12,-34.02,169.49,AV200,BTT,1,1,M,AUTO
79,INFIBEAM,30.06,19.45,7.36,18.61,UP,0.33,-35.30,118.46,AAR,NTT,1,1,L,IT
24,BLUESTARCO,1708.70,800.00,40.63,47.48,UP,-2.85,-53.18,142.47,AAR,NTT,1,1,L,AC


In [58]:
# all prospects
df_prospects_features = df_prospects_features.sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_prospects_features, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
161,YESBANK,25.60,383.47,16.36,22.66,UP,-0.89,1397.93,61.01,NaN,NaN,1,0,L,BANKS
108,NETWORK18,82.92,676.90,-5.77,-184.01,NaN,0.07,716.33,44.46,NaN,NaN,1,0,L,MEDIA
136,SUZLON,54.63,415.22,33.24,-0.54,UP,-0.16,660.06,206.05,NaN,NaN,1,0,L,ELECTRICAL
149,TV18BRDCST,43.22,211.63,-11.80,-949.75,DOWN,0.42,389.66,9.97,AVR,ATH,1,0,M,ENTERTAINMENT
72,IBREALEST,138.47,619.91,27.04,-124.54,UP,-2.18,347.69,120.85,NaN,NaN,1,0,L,REALTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,CASTROLIND,251.58,172.00,41.34,67.95,UP,0.78,-31.63,102.22,AVR,NTT,0,0,M,CHEMICALS
12,ARE&M,1615.75,1066.00,69.90,88.00,NaN,-2.12,-34.02,169.49,AV200,BTT,1,1,M,AUTO
79,INFIBEAM,30.06,19.45,7.36,18.61,UP,0.33,-35.30,118.46,AAR,NTT,1,1,L,IT
24,BLUESTARCO,1708.70,800.00,40.63,47.48,UP,-2.85,-53.18,142.47,AAR,NTT,1,1,L,AC


In [59]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 58.0 seconds
